# Data Download and Preprocessing

## Steps:

1) Load the data, from our csv files. we will be using official data from the FED, and the BEA, aswell as BLS. 
2) We will be using nominal data, rather than real data initially, because we will then be applying the 'cpi-universal' to convert all data to real data.
3) We will then apply pct_change to the data to make it stationary.
4) We then properly label every column and we may further preprocess the data if necessary.

In [16]:
import pandas as pd
import pandas as pd
! pip install fredapi
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from funcs import loading_csv_functions
from funcs.loading_csv_functions import merge_new_data, merge_new_data_and_apply_pct_change, prepare_cpi_data, preprocess_and_merge
from funcs.loading_csv_functions import load_and_process_cpi_data
def name(self) -> any:
    return self.attribute

import os

In [17]:
import pandas as pd
from local_settings import settings
from fredapi import Fred
import requests

# Use the settings dictionary
api_key = settings['api_key']
series_ids = settings['series_ids']
start_date = settings['start_date']
end_date = settings['end_date']
# Base URL for API requests
base_url = 'https://api.stlouisfed.org/fred/series/observations'

# Initialize the FRED API with your API key
fred = Fred(api_key=settings['api_key'])

In [18]:
import pandas as pd
from fredapi import Fred
from local_settings import settings  # Ensure this import correctly brings in your settings

# Initialize the FRED API with the API key from settings
fred = Fred(api_key=settings['api_key'])

# Function to fetch and prepare data
def fetch_data(series_id):
    try:
        print(f"Fetching data for {series_id}")
        data = fred.get_series(series_id, observation_start=settings['start_date'], observation_end=settings['end_date'])
        data.index = pd.to_datetime(data.index)  # Convert index to datetime
        return pd.DataFrame(data, columns=[series_id])
    except Exception as e:
        print(f"Error fetching data for {series_id}: {str(e)}")
        return pd.DataFrame()

# Fetch and store data for each series in a dictionary
data_frames = {}
for series_id in settings['series_ids']:
    data_frame = fetch_data(series_id)
    if not data_frame.empty:
        data_frames[series_id] = data_frame

# Combine all data into a single DataFrame
combined_data = pd.concat(data_frames.values(), axis=1, keys=data_frames.keys())

# Display the combined DataFrame
print(combined_data.tail())


Fetching data for FEDFUNDS
Fetching data for GDP
Fetching data for CPIAUCSL
Fetching data for CUSR0000SAH1
Fetching data for CPILFESL
Fetching data for PCE
Fetching data for PRFI
Fetching data for PNFI
Fetching data for EXPGS
Fetching data for HOUST
Fetching data for DSPI
Fetching data for DGS2
Fetching data for DGS5
Fetching data for DGS10
Fetching data for AAA
Fetching data for BAA
Fetching data for DTWEXBGS
Fetching data for WTISPLC
Fetching data for IMPGS
Fetching data for GCE
Fetching data for FGCE
Fetching data for GDPCTPI
Fetching data for PCEPI
Fetching data for PCEPILFE
Fetching data for BSHCFAW
Error fetching data for BSHCFAW: Bad Request.  The series does not exist.
Fetching data for PAYEMS
Fetching data for UNRATE
Fetching data for INDPRO
Fetching data for CUMFNS
           FEDFUNDS        GDP CPIAUCSL CUSR0000SAH1 CPILFESL      PCE  \
           FEDFUNDS        GDP CPIAUCSL CUSR0000SAH1 CPILFESL      PCE   
2023-12-26      NaN        NaN      NaN          NaN      NaN     

In [19]:
# drop level, to get rid of double headers
combined_data.columns = combined_data.columns.droplevel()

In [20]:
# Show the combined data
display(combined_data.tail())


# # Display the monthly data

display(monthly_data.tail())

,FEDFUNDS,GDP,CPIAUCSL,CUSR0000SAH1,CPILFESL,PCE,PRFI,PNFI,EXPGS,HOUST,...,IMPGS,GCE,FGCE,GDPCTPI,PCEPI,PCEPILFE,PAYEMS,UNRATE,INDPRO,CUMFNS
2023-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-01-01,5.33,28284.498,309.685,392.452,314.438,19033.8,1138.906,3823.986,3081.162,1375.0,...,3927.089,4931.992,1837.28,124.24,121.962,120.909,157560.0,3.7,101.811,76.2948


,FEDFUNDS,GDP,CPIAUCSL,CUSR0000SAH1,CPILFESL,PCE,PRFI,PNFI,EXPGS,HOUST,...,IMPGS,GCE,FGCE,GDPCTPI,PCEPI,PCEPILFE,PAYEMS,UNRATE,INDPRO,CUMFNS
2023-09-30,5.33,NaN,307.288,385.370,310.644,18791.5,NaN,NaN,NaN,1356.0,...,NaN,NaN,NaN,NaN,121.267,119.842,156667.0,3.8,103.2974,77.6538
2023-10-31,5.33,27956.998,307.531,386.675,311.390,18826.8,1100.790,3783.495,3051.660,1376.0,...,3835.394,4871.828,1820.002,123.289,121.309,120.015,156832.0,3.8,102.5683,77.0617
2023-11-30,5.33,NaN,308.024,388.398,312.349,18903.0,NaN,NaN,NaN,1512.0,...,NaN,NaN,NaN,NaN,121.296,120.122,157014.0,3.7,102.9382,77.3646
2023-12-31,5.33,NaN,308.742,389.979,313.209,19013.7,NaN,NaN,NaN,1566.0,...,NaN,NaN,NaN,NaN,121.448,120.305,157304.0,3.7,102.6149,77.2884
2024-01-31,5.33,28284.498,309.685,392.452,314.438,19033.8,1138.906,3823.986,3081.162,1375.0,...,3927.089,4931.992,1837.280,124.240,121.962,120.909,157560.0,3.7,101.8110,76.2948


# First stationarity check

# OLD SCRIPT:


---

# Part 2: Process the Data. 

# Step 1:
# Turning Nominal Values into Real Values

Here, we adjust the values of our data to reflect real values, rather than nominal values. This is done by applying the 'cpi-universal' to our data.

In [21]:
def deflate_nominal_values(df, cpi_col_name, columns_to_deflate):
    """ Deflate nominal data using the CPI index to real values. """
    for col in columns_to_deflate:
        df[col] = df[col] / df[cpi_col_name] * 100  # Use the column name directly
    return df

# Correct usage:
cpi_col_name = 'CPIAUCSL' 
columns_to_deflate = ['GDP', 'PCE', 'PRFI', 'PNFI', 'EXPGS', 'IMPGS', 'GCE', 'FGCE', 'DSPI']

# Now apply the function using the corrected parameter
deflated_df = deflate_nominal_values(combined_data, cpi_col_name, columns_to_deflate)

# Display the first few rows to check the result
display(deflated_df.head())


,FEDFUNDS,GDP,CPIAUCSL,CUSR0000SAH1,CPILFESL,PCE,PRFI,PNFI,EXPGS,HOUST,...,IMPGS,GCE,FGCE,GDPCTPI,PCEPI,PCEPILFE,PAYEMS,UNRATE,INDPRO,CUMFNS
1959-01-01,2.48,1759.152017,29.01,24.5,29.9,1055.153395,97.718028,175.601517,75.542916,1657.0,...,73.753878,403.981386,249.455360,15.205,15.164,15.501,52478.0,6.0,21.9665,80.1973
1959-02-01,2.43,NaN,29.00,24.5,29.9,1067.586207,NaN,NaN,NaN,1667.0,...,NaN,NaN,NaN,NaN,15.179,15.513,52688.0,5.9,22.3966,81.4428
1959-03-01,2.80,NaN,28.97,24.6,30.0,1079.392475,NaN,NaN,NaN,1620.0,...,NaN,NaN,NaN,NaN,15.189,15.531,53014.0,5.6,22.7193,82.4769
1959-04-01,2.96,1803.495514,28.98,24.6,30.0,1077.294686,101.304348,181.970324,75.141477,1590.0,...,77.791580,409.023464,253.899241,15.249,15.219,15.570,53321.0,5.2,23.2032,83.9922
1959-05-01,2.90,NaN,29.04,24.7,30.1,1088.498623,NaN,NaN,NaN,1498.0,...,NaN,NaN,NaN,NaN,15.227,15.589,53550.0,5.1,23.5528,84.9159


# Step 2: 
# Reducing Variance: Applying logarithmic transformation to the data

In [22]:
import numpy as np
import pandas as pd

def apply_log_transformations(df, columns_to_transform):
    """ Apply the 100 * log transformation to specified DataFrame columns. """
    for col in columns_to_transform:
        df[col] = 100 * np.log(df[col])
    return df

# Example usage:
# Define the columns that need the logarithmic transformation
columns_to_transform = ['GDP', 'PCE', 'PRFI', 'PNFI', 'EXPGS', 'IMPGS', 'GCE', 'FGCE', 'HOUST', 'DSPI']

# Assuming 'deflated_df' is your DataFrame after deflation and is now ready for transformations
log_transformed_df = apply_log_transformations(deflated_df, columns_to_transform)

# Display the first few rows to check the result
display(log_transformed_df.head())

,FEDFUNDS,GDP,CPIAUCSL,CUSR0000SAH1,CPILFESL,PCE,PRFI,PNFI,EXPGS,HOUST,...,IMPGS,GCE,FGCE,GDPCTPI,PCEPI,PCEPILFE,PAYEMS,UNRATE,INDPRO,CUMFNS
1959-01-01,2.48,747.258716,29.01,24.5,29.9,696.144143,458.208607,516.821732,432.470092,741.276402,...,430.073358,600.136880,551.927998,15.205,15.164,15.501,52478.0,6.0,21.9665,80.1973
1959-02-01,2.43,NaN,29.00,24.5,29.9,697.315550,NaN,NaN,NaN,741.878088,...,NaN,NaN,NaN,NaN,15.179,15.513,52688.0,5.9,22.3966,81.4428
1959-03-01,2.80,NaN,28.97,24.6,30.0,698.415364,NaN,NaN,NaN,739.018143,...,NaN,NaN,NaN,NaN,15.189,15.531,53014.0,5.6,22.7193,82.4769
1959-04-01,2.96,749.748201,28.98,24.6,30.0,698.220826,461.812933,520.384362,431.937270,737.148930,...,435.403320,601.377252,553.693750,15.249,15.219,15.570,53321.0,5.2,23.2032,83.9922
1959-05-01,2.90,NaN,29.04,24.7,30.1,699.255462,NaN,NaN,NaN,731.188616,...,NaN,NaN,NaN,NaN,15.227,15.589,53550.0,5.1,23.5528,84.9159


In [26]:
import pandas as pd
df = log_transformed_df 
# Assuming 'df' is your DataFrame and the index is datetime
df.index = pd.to_datetime(df.index)  # Ensure the index is in datetime format

# Filter data to only include the first day of each month
monthly_data = df[df.index.is_month_start]

# Display the resulting DataFrame
display(monthly_data)

# save as csv 'economic_data.csv
df = monthly_data
df.to_csv('economic_data.csv')

,FEDFUNDS,GDP,CPIAUCSL,CUSR0000SAH1,CPILFESL,PCE,PRFI,PNFI,EXPGS,HOUST,...,IMPGS,GCE,FGCE,GDPCTPI,PCEPI,PCEPILFE,PAYEMS,UNRATE,INDPRO,CUMFNS
1959-01-01,2.48,747.258716,29.010,24.500,29.900,696.144143,458.208607,516.821732,432.470092,741.276402,...,430.073358,600.136880,551.927998,15.205,15.164,15.501,52478.0,6.0,21.9665,80.1973
1959-02-01,2.43,NaN,29.000,24.500,29.900,697.315550,NaN,NaN,NaN,741.878088,...,NaN,NaN,NaN,NaN,15.179,15.513,52688.0,5.9,22.3966,81.4428
1959-03-01,2.80,NaN,28.970,24.600,30.000,698.415364,NaN,NaN,NaN,739.018143,...,NaN,NaN,NaN,NaN,15.189,15.531,53014.0,5.6,22.7193,82.4769
1959-04-01,2.96,749.748201,28.980,24.600,30.000,698.220826,461.812933,520.384362,431.937270,737.148930,...,435.403320,601.377252,553.693750,15.249,15.219,15.570,53321.0,5.2,23.2032,83.9922
1959-05-01,2.90,NaN,29.040,24.700,30.100,699.255462,NaN,NaN,NaN,731.188616,...,NaN,NaN,NaN,NaN,15.227,15.589,53550.0,5.1,23.5528,84.9159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,5.33,NaN,307.288,385.370,310.644,871.854469,NaN,NaN,NaN,721.229447,...,NaN,NaN,NaN,NaN,121.267,119.842,156667.0,3.8,103.2974,77.6538
2023-10-01,5.33,911.501711,307.531,386.675,311.390,871.963096,588.037767,711.499776,690.003527,722.693602,...,712.862174,736.781880,638.318717,123.289,121.309,120.015,156832.0,3.8,102.5683,77.0617
2023-11-01,5.33,NaN,308.024,388.398,312.349,872.206840,NaN,NaN,NaN,732.118856,...,NaN,NaN,NaN,NaN,121.296,120.122,157014.0,3.7,102.9382,77.3646
2023-12-01,5.33,NaN,308.742,389.979,313.209,872.557926,NaN,NaN,NaN,735.627988,...,NaN,NaN,NaN,NaN,121.448,120.305,157304.0,3.7,102.6149,77.2884


# Missing Data: (ADD LATER)

## Stock Price Index: Standard & Poor’s 500 Composite: 
## S&P GSCI Non-Energy Commodities Nearby Index: 
## S&P 500 VOLATILITY INDEX: VIX